In [ ]:
# ==============================
# 1. 운영체제(OS) 관련 기능 사용
# ==============================
import os
# → 환경변수(API KEY 등)를 읽기 위해 사용

# ==============================
# 2. .env 파일 읽기 위한 라이브러리
# ==============================
from dotenv import load_dotenv
# → .env 파일에 저장된 값을 파이썬 환경변수로 불러오는 함수


# ==============================
# 3. .env 파일 로드
# ==============================
load_dotenv(override=True)


# → .env 파일 내용을 실제로 메모리에 로드
# → override=True : 이미 같은 이름의 환경변수가 있어도 .env 값으로 덮어쓰기


# ==============================
# 4. OpenAI API KEY 가져오기
# ==============================
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
# → 환경변수에 저장된 OPENAI_API_KEY 값을 읽어서
# → 파이썬 변수 OPENAI_API_KEY에 저장
# → 이후 OpenAI, LangChain 모델 사용 시 인증용으로 사용됨




In [ ]:
# ==============================
# 5. 텍스트 파일 로더 불러오기
# ==============================
from langchain_classic.document_loaders import TextLoader
# → 텍스트 파일(.txt)을 LangChain의 Document 형식으로 변환하는 도구

# ==============================
# 6. AI.txt 파일 불러오기
# ==============================
documents=TextLoader("AI.txt").load()

# → AI.txt 파일을 읽어서
# → LangChain Document 객체 리스트 형태로 변환
# → 결과 예시:
#   [Document(page_content="파일 내용", metadata={"source":"AI.txt"})]

In [ ]:
# ==============================
# 7. 텍스트 분할기 불러오기
# ==============================
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
# → 긴 문서를 작은 조각(chunk)으로 나누는 도구
# → 문단 → 문장 → 글자 순서로 자연스럽게 분할


# ==============================
# 8. 문서 분할 함수 정의
# ==============================
def split_docs(documents,chunk_size=1000,chunk_overlap=20):
     # 텍스트 분할기 객체 생성
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size, # 한 chunk 당 최대 길이
                                                 chunk_overlap=chunk_overlap)# chunk 사이 겹치는 부분
    
    # 실제 문서 분할 수행
    docs=text_splitter.split_documents(documents)
    # → 긴 문서를 여러 개의 작은 Document로 나눔
    # → 결과 예시:
    #   [Document(chunk1), Document(chunk2), Document(chunk3)...]

    # 분할된 문서 반환
    return docs


# ==============================
# 9. 함수 실행
# ==============================
docs=split_docs(documents)

# → AI.txt에서 불러온 documents를
# → 1000자 단위로 나누고
# → 결과를 docs 변수에 저장


In [ ]:
# ==============================
# 1. OpenAI 임베딩 모델 불러오기
# ==============================
from langchain_openai import OpenAIEmbeddings
# → OpenAI에서 제공하는 "텍스트 → 숫자 벡터" 변환 클래스

# ==============================
# 2. 임베딩 객체 생성
# ==============================
embeddings=OpenAIEmbeddings(model='text-embedding-ada-002', # 텍스트를 벡터로 변환하는 OpenAI 임베딩 모델
                            api_key=OPENAI_API_KEY)# OpenAI API 인증 키


# → embeddings 객체 역할:
#    문자열 → 의미 기반 숫자 벡터로 변환
#    예: "AI란?" → [0.23, -0.11, 1.02, ...]


# ==============================
# 3. 벡터 데이터베이스(Chroma) 불러오기
# ==============================

from langchain_classic.vectorstores import Chroma
# → 문서 벡터를 저장하고 빠르게 검색할 수 있는 벡터 DB

# ==============================
# 4. 문서를 벡터로 변환하여 DB에 저장
# ==============================
db=Chroma.from_documents(docs,  # 이전 단계에서 쪼갠 문서 리스트
                         embeddings,# 임베딩 모델 (텍스트 → 벡터 변환기)
                         persist_directory='data')# 벡터 DB를 저장할 폴더 이름

In [8]:
# 내부에서 자동으로 일어나는 과정:
# docs (문서 텍스트)
#      ↓
# embeddings (벡터 변환)
#      ↓
# Chroma DB 저장
#
# 결과:
# db 변수에 "검색 가능한 벡터 데이터베이스" 생성됨


# ==============================
# 5. GPT 모델 불러오기
# ==============================

from langchain_classic.chat_models import ChatOpenAI
# → OpenAI GPT 모델을 LangChain에서 사용하기 위한 클래스


# ==============================
# 6. 사용할 GPT 모델 이름 설정
# ==============================
model_name='gpt-4.1-mini'
# → 빠르고 비용 효율적인 GPT 모델

# ==============================
# 7. LLM 객체 생성 (GPT 연결)
# ==============================
llm=ChatOpenAI(model_name=model_name,# 사용할 GPT 모델
               api_key=OPENAI_API_KEY)# OpenAI API 인증 키


# → 이제 llm 객체를 통해 GPT에게 질문 가능


# ==============================
# 8. 질문-답변 체인 불러오기
# ==============================
from langchain_classic.chains.question_answering import load_qa_chain
# → 문서 + 질문을 묶어서 GPT에게 전달하는 자동 파이프라인


# ==============================
# 9. QA 체인 생성
# ==============================
chain=load_qa_chain(llm,# 사용할 GPT 모델
                    chain_type='stuff',# 검색된 문서를 모두 한 번에 GPT에게 전달
                    verbose=True) # 내부 처리 과정을 콘솔에 출력 (디버깅용)

# chain_type='stuff' 의미:
# → 여러 문서를 하나로 합쳐서 GPT에게 그대로 전달
# → 소규모 문서일 때 가장 간단한 방식


# ==============================
# 10. 사용자 질문 설정
# ==============================
query="AI를 한단어로 말한다면?"
# → 사용자가 입력한 질문


# ==============================
# 11. 벡터 DB에서 유사 문서 검색
# ==============================
matching_docs=db.similarity_search(query)

# 내부 처리 과정:
# query ("AI란?")
#      ↓
# 임베딩 변환
#      ↓
# DB에 저장된 문서 벡터와 비교
#      ↓
# 의미적으로 가장 비슷한 문서 반환

# 결과:
# matching_docs = [Document1, Document2, ...]


# ==============================
# 12. GPT에게 문서 + 질문 전달 후 답변 생성
# ==============================
answer=chain.run(input_documents=matching_docs, # 검색된 관련 문서들
                 question=query) # 실제 질문

# 내부 흐름:
# 문서 + 질문
#      ↓
# 프롬프트 자동 생성
#      ↓
# GPT 호출
#      ↓
# 답변 생성


# ==============================
# 13. 최종 결과 출력
# ==============================
answer
# → AI가 생성한 최종 답변 출력



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Artificial intelligence (AI) is the intelligence of machines or software, as opposed to the intelligence of humans or animals. It is a field of study in computer science that develops and studies intelligent machines. Such machines may be called AIs.

AI technology is widely used throughout industry, government, and science. Some high-profile applications are: advanced web search engines (e.g., Google Search), recommendation systems (used by YouTube, Amazon, and Netflix), understanding human speech (such as Google Assistant, Siri, and Alexa), self-driving cars (e.g., Waymo), generative and creative tools (ChatGPT and AI art), and superhuman play and analysis in strategy games (such as chess and Go).[1

'AI를 한 단어로 표현한다면 "지능"이라고 할 수 있습니다.'